# 8 正则表达式

**处理原始物理数据文本可能有下列操作：**
·搜索和更正不规则的地方。
·在数百个文件中查找和替换文本。
·求值数学表达式。
·处理数字格式。
·调整表格中的数据。

**基本的正则表达式语法应用：**
·更有效地浏览命令行。
·根据命令行的内容快速查找文件（grep）。
·同时在多个文件中查找和替换复杂表达式（sed）。
·快速对来自命令行的纯文本数据列执行数学运算（awk）。

## 8.1 应用介绍

正则表达式是进行文本匹配的工具。
命令行脚本工具（如grep、sed、awk、Python和Perl）都使用正则表达式。

计算物理中自动地修改大文件和文件集。
使用元字符进行匹配

## 8.2 命令行中的元字符

元字符是除了其字面意义之外具有特殊意义的字符。
字面字符是仅使用字面意义的字符。
正则表达式就是由元字符和字面字符组成的字符串。

**元字符遵循以下规则：**
·字母数字会匹配自身。
·点（.）匹配任何字符。
·<code>*、+</code>和<code>？</code>用来匹配重复的模式。
·字符集<code>[]</code>和或运算符<code>|</code>用于选择性匹配。
·位置标记<code>^</code>和<code>&dollar;</code>和分别匹配行的行首和行末。
·括号用来分组，并从匹配中提取信息。

### 列出文件（命令行）

#### 通配符：*

表示与该字符之前的字符匹配0次或多次。
如“zo*”表示“所有含有‘zo’的字符串”

<code>ls *.dat</code>列出所有以0个或多个任意字符开头、后跟.dat字符串的文件。
这样使用存在一些问题：
1.ls命令只能进入目录树的第一层。
2.使用其他扩展名的数据文件无法列出。

### 根据模式全局查找文件名：find（正则）

find命令有许多种使用方式，其中一种是将其与正则表达式结合起来：<code>find [path] -regex "\<expression\>"</code>
通过<path>手动规定搜索路径。然后用-regex选项指不规则的表达式定一个位于双引号中的正则表达式，以此来递归解析文件名，寻找匹配的内容。
如：<code>find -regex ".*\.dat"</code>这样可以搜索子目录

#### 表示任意字符：.

正则表达式中的点号可以表示任何**单个**字符。

#### 转义元字符（\）

要匹配的模式中有时包含了实际的点号（.）或星号（*）。如果要表示这是一个字面值字符，而不是元字符，必须将其转义。
转义元字符不使用字符的元含义（meta-meaning）。

用反斜杠（\）来转义元字符。
元字符有两个含义，分别是特殊含义和字面含义。反斜杠用来在两者之间切换。

#### 匹配多个模式：|

为了匹配具有各种扩展名（.txt、.dat、.data、.DAT）的文件，需要一个“或”布尔操作。
在正则表达式中称为替代（alternation），语法为（|）。

#### 字符集：\[…]

字符集[]或[{}]语法用来匹配一组字符，能够匹配其中任意一个字符。
很容易就能避免因大小写、不同的拼写方式、以及其他原因导致的匹配问题。

<code>find . -regax ".*\.[Dd][Aa][Tt][Aa]"</code>当前目录及子目录中所有扩展名中含有字符集中的字母的文件。

## 8.3 grep、sed、awk

·grep命令的基本语法为：<code>grep \<pattern\>\<inputfile\></code>。用来抓取匹配的模式并打印出来。
·sed命令的基本语法为：<code>sed "s/\<pattern\>/\<substitution\>/"
\<inputfile\></code>。sed相当于带有替换功能的grep。
awk命令的基本语法为：<code>awk pattern [action]</code>。awk用来处理列文本。

## 8.4 在文件中查找模式：grep

grep的工作方式很像Google，根据正则表达式全局搜索文件的内容。

<code>grep Gs *</code>：在目录中的文件中搜索所有的Gs实例。

**常用option：**
-r：递归标志。
-c：计算匹配的行数。
-i：忽略大写。

## 8.5 在文件中查找和替换：sed

sed是一个替换工具。
在grep基础上多了一个功能：匹配上的一个字符串可以替换为其他字符串。
sed可以看作是grep的扩展版。

语法：<code>sed "s/\<expression\>/\<substitution\>/g" \<inputfile\> </code>

s（第一个斜杠之前）表示“替换”，g（最后一个斜杠之后，意为globally）表示在整个文件中进行替换。如果没有g，sed只会替换每行中匹配上的第一个实例。

如：<code>sed "s/2014\.05\.01/2014-05-01/g" atmos_sat_42.dat</code>
保存输出到文件：<code>sed "s/\<expression\>/\<substitution\>/" \<oldfile\>\>\<newfile\></code>
原地编辑文件：<code>sed -i "s/\<expression\>/\<substitution\>/" \<oldfile\></code>
管道连接sed的输入和输出：<code>sed "s/a/A/" oldfile.txt | sed "s/b/B/"</code>

### 查找和替换复杂模式

针对需要保存的值，sed提供了搜索过程中的捕获模式。
需要重用匹配模式中的某一部分，保存匹配：<code>\(x\)</code>，记住捕获括号标记的匹配x。

匹配模式：<code>"20[01][0-9].&ast;[0-9][0-9].&ast;[0-9][0-9]"</code>
匹配“YYYY-MM-DD”的时间。年份20开头，十位为0或1，个位为0到9。月份和日期均为0到9.

修复格式错误的日期<code>sed "s/\(20[01][0-9]\).&ast;\([0-9][0-9]\).&ast;\([0-9][0-9]\)/\1-\2-\3/g" \<filename.dat\></code>

### sed额外内容

**文件操作和格式化**
使用d字符删除文件中的所有空行：<code>sed '/&circumflex;\&dollar;/d' data.txt</code>
将文件长度翻倍的暴力方式：<code>sed -e '/&circumflex;&dollar;/d' -e 's/&circumflex;\(.\+\)&dollar;/\1\n/' data.txt</code>
将文件长度翻倍的G字符方式：<code>sed -e '/&circumflex;&dollar;/d' -e G data.txt</code>
更简单的：<code>sed '/&circumflex;&dollar;/d;G' data.txt</code>
重新格式化数据。用到了字符串记忆功能“()”：<code>sed 's/.&ast;\([0-9]\{3\}\).&ast;\([0-9]\{3\}\).&ast;\([0-9]\{3\}\).&ast;/(\1)\2-\3/' data.txt</code>

**行号**
显示行号：<code>sed '/./=' wordlist.txt | sed '/./N; s/\n/ /'</code>
只修改文件的一部分，在命令开头用行号来指明要修改的行：<code>sed '4 s/r/R/' wordlist.txt</code>
用两个数字（用逗号分隔）指定要修改的行号范围：<code>sed 'sed '4,6 s/r/T/' wordlist.txt</code>
用模式匹配来选择要修改的行：<code>sed '/^z/ s/$/zzzzzzzzzz$/' wordlist.txt
sed '/wade/,/salt/ s/m/PPPPPPPPPP/' wordlist.txt</code>

**正则表达式示例**
|表达式| 匹配|

|---|---|
|uvwxyz| uvwxyz|
|[u-z] |u、v、q、x、y、z中的一个|
|[^ ] |除空格之外的任意一个字符|
|p\*i |0个或多个p字符，后面跟着1个i字符，如pi、ppppi、i |
|.\* |0个或多个任意字符，如super califragilisticexpialidocious、42|
|\|\n\|^spelunking.(.)|以spelunking起始的行，后面跟着一对括号，括号中可以含有任意内容|
|\$ |以反斜杠结尾的行|
|\$ |美元符号|
|.{4}z |任意4个字符，最后跟着字符z|

**正则表达式可用于：**
·根据文件名称查找文件（find）。
·根据文件的内容查找文件（grep）。
·根据找到的模式替换内容（sed）。

## 8.6 处理数据列（awk）

物理学中的很多数据的格式都只是纯文本文档中的数字列。awk用于处理此类数据。

### 基本处理
<code>awk '/144/' rgb.txt</code>：返回所有含144的行
<code>awk '/&circumflex;144/' rgb.txt</code>：从每一行的开始处搜索144
<code>awk '/&circumflex;./144\./' rgb.txt</code>：从每一行的中间搜索144

### 列处理
<code>awk '{print &dollar;1&dollar;2&dollar;3}' rgb.txt</code>：打印1、2、3列
<code>awk '/&circumflex;255/{print &dollar;1&dollar;2&dollar;3}' rgb.txt</code>：从1、2、3列的开头搜索255
<code>awk '/&circumflex;.+ +.+144/{print &dollar;1&dollar;2&dollar;3}' rgb.txt</code>：从指定位置搜索144
<code>awk '{print &dollar;1," ",&dollar;2," ",&dollar;2," "&dollar;1}' rgb.txt</code>：交换1,2列的位置
<code>awk NR==11'{print &dollar;1 &dollar;2 &dollar;3,"\t",&dollar;4}' rgb.txt</code>：修改一行的文本
<code>awk NR==11'{print &dollar;1,"+",&dollar;2,"+",&dollar;3,"=",&dollar;1+&dollar;2+&dollar;3}' rgb.txt</code>：进行运算

## 8.7 Python正则表达式

Python正则表达式模块re中有以上工具的替代品，re是Python标准库的一部分。
re模块使用Python风格的正则表达式进行模式匹配。将re与其他Python模块的强大功能结合之后，就能在Python中重现grep、sed、awk的功能。

### grep的功能
<code>re.match(<pattern>, <string>)</code>在字符串的开头匹配正则表达式。
<code>re.search(<pattern>, <string>)</code>在字符串中搜索匹配的模式。
<code>re.findall(<pattern>, <string>)</code>查找字符串中所有匹配的模式。

### sed的功能
<code>re.sub(<pattern>, <replacement>, <string>)替换字符串中所有出现的模式。
<code>re.subn(<pattern>, <replacement>, <string>)替换字符串中所有出现的模式，并返回替换的数量。

### re模型的其它实用程序
<code>re.split(pattern, string)</code>根据出现的模式来分割字符串。
<code>re.finditer(pattern, string)</code>返回一个迭代器，每次匹配时产生一个匹配对象。
<code>re.compile(pattern)</code>预编译正则表达式，加快后续匹配的速度。

如果找不到与正则表达式相符的匹配，则返回None。
如果找到匹配，则返回特殊的MatchObject。
MatchObject有方法和属性，用于确定匹配在字符串中的位置、原始的正则表达式以及用括号捕捉位于MatchObject.groups()方法中的值。

In [2]:
import re


pattern = "20[01][0-9].*[0-9][0-9].*[0-9][0-9]"
re.match(pattern, '2015-12-16')
m = re.match(pattern, '2015-12-16')
m.pos
m.groups()
m = re.match("20[01][0-9].*[0-9][0-9].*[0-9][0-9]", 'not-a-date')
print(m is None)

AttributeError: 're.Match' object has no attribute 'pattern'